**Load the required libraries and data**

In [1]:
# Load the libraries
import pandas as pd 
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

In [3]:
# Load the data
May = pd.read_excel("data/May.xlsx")
June = pd.read_excel("data/June.xlsx")
July = pd.read_excel("data/July.xlsx")

# Explanatory Data Analysis
## Data Inspection and Cleaning 
View the first five rows of each dataset

In [4]:
May.head()

,Title,Price,Area,Bed,Bath,Furnished
0,"2 Bed Flat, Turner House, WC2R","£5,000 per month",0.43 km,2.0,2,Furnished
1,"1 Bed Flat, Bank Chambers, SW1Y","£2,578 per month",0.56 km,1.0,1,Furnished
2,"Studio Flat, Shaftesbury Avenue, W1D","£2,000 per month",0.57 km,NaN,1,Furnished
3,"2 Bed Flat, Rupert Street, W1D","£3,000 per month",0.65 km,2.0,2,Furnished
4,"1 Bed Flat, Old Compton Street, W1D","£1,950 per month",0.73 km,1.0,1,Furnished


In [5]:
June.head()

,Title,Price,Area,Bed,Bath,Furnished
0,"2 Bed Flat, Villiers Street, WC2N","£2,600 per month",0.29 km,2.0,1,Furnished
1,"2 Bed Flat, Bedfordbury, WC2N","£8,333 per month",0.4 km,2.0,2,Furnished
2,"1 Bed Flat, Bedford Street, WC2E","£2,362 per month",0.42 km,1.0,1,Unfurnished
3,"2 Bed Flat, Turner House, WC2R","£6,000 per month",0.43 km,2.0,2,Furnished
4,"Studio Flat, Shaftesbury Avenue, W1D","£1,950 per month",0.57 km,NaN,1,Furnished


In [6]:
July.head()

,Title,Price,Area,Bed,Bath,Furnished
0,"Room in a Shared Flat, London, E14","£1,100 per month",0.02 km,NaN,1,Furnishing at tenant choice
1,"3 Bed Flat, Norway House, SW1Y","£6,500 per month",0.15 km,3.0,2,Furnished
2,"Studio Flat, Craven Street, WC2N","£1,800 per month",0.23 km,NaN,1,Furnished
3,"1 Bed Flat, Craven Street, WC2N","£1,800 per month",0.23 km,1.0,1,Furnished
4,"1 Bed Flat, Oxendon Street, SW1Y","£2,470 per month",0.34 km,1.0,1,Furnished
